In [1]:
import numpy as np
import pandas as pd
import sklearn 
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

In [2]:
import imblearn 
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss   # have not used it
from collections import Counter

In [3]:
data = pd.read_csv("D:\\Downloads\\train_yaOffsB.csv")

In [4]:
df = data.iloc[:,1:]

In [5]:
df.head()

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
0,188,1,0,1,0,0.0,0,1,0
1,209,1,0,1,0,0.0,0,2,1
2,257,1,0,1,0,0.0,0,2,1
3,257,1,1,1,0,0.0,0,2,1
4,342,1,0,1,0,0.0,0,2,1


In [6]:
df.isnull().sum()

Estimated_Insects_Count       0
Crop_Type                     0
Soil_Type                     0
Pesticide_Use_Category        0
Number_Doses_Week             0
Number_Weeks_Used          9000
Number_Weeks_Quit             0
Season                        0
Crop_Damage                   0
dtype: int64

In [7]:
df.Number_Weeks_Used.isnull().sum()

9000

In [8]:
df.Number_Weeks_Used[df.Number_Weeks_Used.isnull()] = df.Number_Weeks_Used[df.Number_Weeks_Used>5].median()

C:\Users\ethic\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
df.Number_Weeks_Used[df.Number_Doses_Week==0] = 0

C:\Users\ethic\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
df.Number_Weeks_Used.isnull().sum()

0

In [11]:
x = df.iloc[:,0:8]
y = df[["Crop_Damage"]]

In [12]:
samp = SMOTE()
x_res,y_res=samp.fit_sample(x,y)
samp_1 = NearMiss()
x_res1,y_res1 = samp_1.fit_sample(x_res,y_res)

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x_res1, y_res1, test_size=0.3)

In [14]:
class_weights = { 0:1.3293714209615128, 1:8.029964806435395, 2:51.28965960179833}

In [15]:
clf=RandomForestClassifier(class_weight=class_weights)

In [16]:
clf.fit(x_train,y_train)
y_pred_random=clf.predict(x_test)

C:\Users\ethic\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


In [17]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_random))

Accuracy: 0.8329417047070269


In [18]:
metrics.confusion_matrix(y_test,y_pred_random)

array([[18303,  3124,   824],
       [ 2943, 17265,  2112],
       [  771,  1388, 20085]], dtype=int64)

In [19]:
from xgboost import XGBClassifier

In [20]:
xgb = XGBClassifier(objective='multi:softprob', n_estimators = 500, weight = class_weights)

In [21]:
xgb.fit(x_train,y_train)

C:\Users\ethic\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None,
              weight={0: 1.3293714209615128, 1: 8.029964806435395,
                      2: 51.28965960179833})

In [22]:
y_pred_xgb=xgb.predict(x_test)

In [23]:
metrics.accuracy_score(y_test, y_pred_xgb)

0.8262515902117787

In [24]:
metrics.confusion_matrix(y_test,y_pred_xgb)

array([[20499,  1488,   264],
       [ 3023, 15215,  4082],
       [  559,  2193, 19492]], dtype=int64)

In [25]:
from sklearn.neighbors import KNeighborsClassifier

In [26]:
knn = KNeighborsClassifier(n_neighbors=5 )

In [27]:
knn.fit(x_train,y_train)

C:\Users\ethic\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


KNeighborsClassifier()

In [28]:
y_pred_knn = knn.predict(x_test)

In [29]:
metrics.accuracy_score(y_test, y_pred_knn)

0.7853027014891866

In [30]:
metrics.confusion_matrix(y_test,y_pred_knn)

array([[14608,  5609,  2034],
       [ 2108, 18025,  2187],
       [  637,  1770, 19837]], dtype=int64)

In [31]:
from sklearn.tree import DecisionTreeClassifier 

In [32]:
tree = DecisionTreeClassifier(max_depth = 5,class_weight= class_weights)

In [33]:
tree.fit(x_train,y_train)

DecisionTreeClassifier(class_weight={0: 1.3293714209615128,
                                     1: 8.029964806435395,
                                     2: 51.28965960179833},
                       max_depth=5)

In [34]:
y_pred_tree=tree.predict(x_test)

In [35]:
metrics.accuracy_score(y_test, y_pred_tree)

0.380692958168076

In [36]:
metrics.confusion_matrix(y_test,y_pred_tree)

array([[ 1776,  1785, 18690],
       [  155,  1507, 20658],
       [   13,    78, 22153]], dtype=int64)

In [37]:
tree1 = DecisionTreeClassifier(max_depth = 25,class_weight= class_weights)

In [38]:
tree1.fit(x_train,y_train)

DecisionTreeClassifier(class_weight={0: 1.3293714209615128,
                                     1: 8.029964806435395,
                                     2: 51.28965960179833},
                       max_depth=25)

In [39]:
y_pred_tree1 = tree1.predict(x_test)

In [40]:
metrics.accuracy_score(y_test, y_pred_tree1)

0.7553693033001572

In [41]:
metrics.confusion_matrix(y_test,y_pred_tree1)

array([[15690,  4298,  2263],
       [ 2824, 13992,  5504],
       [  628,   828, 20788]], dtype=int64)

In [42]:
from sklearn.ensemble import VotingClassifier

In [43]:
estimators=[('knn', knn), ('tree1', tree1), ('xgb', xgb),('clf', clf),('tree', tree)]

In [44]:
ensemble = VotingClassifier(estimators=estimators)

In [ ]:
ensemble.fit(x_train, y_train)

C:\Users\ethic\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [ ]:
y_pred = ensemble.predict(x_test)

In [ ]:
metrics.accuracy_score(y_test, y_pred)

In [ ]:
metrics.confusion_matrix(y_test,y_pred)

In [ ]:
test = pd.read_csv("D:\\Downloads\\test_pFkWwen.csv")

In [ ]:
x_1=test.drop(["Crop_Damage"],axis=1)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['Pesticide_Use_Category','Season'])], remainder='passthrough')
x_1=np.array(ct.fit_transform(x_1))
x_1.shape